In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from skimage.color  import rgb2ycbcr 
from skimage.feature import local_binary_pattern
from scipy.fftpack import  dct
from sklearn.decomposition import PCA
import skimage.io as io
import numpy as np
import progressbar
import glob
import cv2
import tensorflow as tf


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [30]:
def extract_lbp_dct(blocks):
    n_points = 8
    radius = 1
    # Extract feature vector from given blocks
    # Input: List of blocks response with given image
    # Output: Feature vector of given image
    n_blocks, block_size, _, _ = blocks.shape
    CR_feature = np.zeros((n_blocks, block_size, block_size))
    CB_feature = np.zeros((n_blocks, block_size, block_size))
    for idx, block in enumerate(blocks):
        CR_lbp          = local_binary_pattern(block[:, :, 0], n_points, radius)
        CR_lbp          = np.float32(CR_lbp)
        CR_feature[idx] = dct(CR_lbp)
        
        CB_lbp          = local_binary_pattern(block[:, :, 1], n_points, radius)
        CB_lbp          = np.float32(CB_lbp)
        CB_feature[idx] = dct(CB_lbp)
    CR_feature = np.std(CR_feature, axis = 0).flatten()
    CB_feature = np.std(CB_feature, axis = 0).flatten()
    return np.concatenate([CR_feature, CB_feature], axis = 0)


In [50]:
def extract_feature(authentic_list, forged_list):
    # Read and extract feature vector from given list images
    block_sizes = [8]
    strides = [16]
    
    Y_train = np.zeros((len(authentic_list) + len(forged_list), ), dtype = np.float32)
    Y_train[: len(authentic_list)] = 1.0
    X_train=[]
    list_img= authentic_list + forged_list
    total_img = len(authentic_list) + len(forged_list)
    dim = 0
    for i in range(len(block_sizes)):
        dim += block_sizes[i] ** 2
    features = np.zeros((total_img, 2*dim))
    for idx in progressbar.progressbar(range(total_img)):
        im         = list_img[idx]
        #bgr_img    = io.imread(im)
        #extract chromatic channel
        #ycrcb_image = rgb2ycbcr(bgr_img[:,:,:3])  
        bgr_img    = cv2.imread(im)
        ycrcb_image = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2YCR_CB)
        ycrcb_image=ycrcb_image[:, :, 1:]
        
        #img_hsv = convert_colorspace(bgr_img, 'RGB', 'YCbCr')
        #ycrcb_image=img_hsv[:, :, 1:]
        tmp        = 0
        for k, bz in enumerate(block_sizes):
            stride=strides[k]
             
            #block processing
            height, width, _ = ycrcb_image.shape
            img_blocks = []
            for i in range(0, height - bz, stride):
                for j in range(0, width - bz, stride):
                    img_blocks.append(ycrcb_image[i: i + bz, j: j + bz])
                    
            img_blocks=np.array(img_blocks)
            features[idx, tmp: tmp + 2*bz**2] = extract_lbp_dct(img_blocks)
            tmp += 2*bz ** 2
        X_train=features
    return X_train, Y_train

In [51]:
with tf.device('/GPU'):
    if __name__ == '__main__':
        authentic_folder = ['CASIA2/Au/*.jpg']
        forged_folder = ['CASIA2/Tp/*.jpg', 'CASIA2/Tp/*.tif']
        print('Fetature Extraction Starting...')
        authentic_list = []
        forged_list = []
        for au_img in authentic_folder:
            authentic_list += glob.glob(au_img)
        for tp_img in forged_folder:
            forged_list += glob.glob(tp_img)
        X,Y=extract_feature(authentic_list,forged_list)
        X, Y = shuffle(X, Y)
        print('Fetature Extraction done')

Fetature Extraction Starting...


100% (12560 of 12560) |##################| Elapsed Time: 1:11:32 Time:  1:11:32


Fetature Extraction done


In [87]:
np.shape(X_train)

(12560, 128)

In [85]:
#reatain 96% of original data
pca = PCA(.96)
X_pca=X_train
pca.fit(X_pca)
X_pca = pca.transform(X_pca)

In [86]:
np.shape(X_pca)

(12560, 25)

In [82]:
(trainX, testX, trainY, testY) = train_test_split(X_pca, Y,test_size=0.20, stratify=Y, random_state=42)
svclassifier = SVC(kernel='rbf')
svclassifier.fit(trainX, trainY)
predY = svclassifier.predict(testX)
#confusion Matricx and classification report
print(confusion_matrix(testY,predY))
print(classification_report(testY,predY))

[[1006   19]
 [  64 1423]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      1025
         1.0       0.99      0.96      0.97      1487

    accuracy                           0.97      2512
   macro avg       0.96      0.97      0.97      2512
weighted avg       0.97      0.97      0.97      2512



In [89]:
print('Building SVM Model...')
scaler  = StandardScaler()
X_train = scaler.fit_transform(X)
clf = LinearSVC(dual=False)
#cross-validated with 10 fold
scores = cross_val_score(clf,X_train,Y ,cv=10, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print('Building SVM Model Done.')

Building SVM Model...
Accuracy: 0.96 (+/- 0.01)
Building SVM Model Done.


In [91]:

trainX_cnn=trainX
testX_cnn=testX
trainY_cnn=trainY
testY_cnn=testY
#(trainX, testX, trainY, testY) = train_test_split(trainX, trainY,test_size=0.20, stratify=trainY, random_state=42)

trainX_cnn = trainX_cnn.reshape(trainX_cnn.shape[0],5,5,1) 
testX_cnn = testX_cnn.reshape(testX_cnn.shape[0],5,5,1) 
print(np.shape(trainX_cnn), np.shape(testX_cnn))

trainY_cnn = trainY_cnn.reshape(trainY_cnn.shape[0],1) 
testY_cnn = testY_cnn.reshape(testY_cnn.shape[0],1) 
print(np.shape(trainY_cnn), np.shape(testY_cnn))

trainY_cnn = tf.keras.utils.to_categorical(trainY_cnn)
testY_cnn = tf.keras.utils.to_categorical(testY_cnn)
print(np.shape(trainY_cnn), np.shape(testY_cnn))


(10048, 5, 5, 1) (2512, 5, 5, 1)
(10048, 1) (2512, 1)
(10048, 2) (2512, 2)


In [93]:
# Create a convolutional neural network
model = tf.keras.models.Sequential([

    # Convolutional layer. Learn 32 filters using a 3x3 kernel
    tf.keras.layers.Conv2D(
        32, (3, 3), activation="relu", input_shape=(5,5,1)
    ),

    # Max-pooling layer, using 2x2 pool size
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten units
    tf.keras.layers.Flatten(),

    # Add a hidden layer with dropout
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),

    # Add an output layer with output units for all  2 output
    tf.keras.layers.Dense(2, activation="softmax")
])


In [95]:
# Train neural network
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(trainX_cnn, trainY_cnn, epochs=20)

Epoch 1/20
314/314 [==============================] - 4s 10ms/step - loss: 0.0950 - accuracy: 0.9624
Epoch 2/20
314/314 [==============================] - 3s 10ms/step - loss: 0.0895 - accuracy: 0.9637
Epoch 3/20
314/314 [==============================] - 3s 10ms/step - loss: 0.0934 - accuracy: 0.9625
Epoch 4/20
314/314 [==============================] - 3s 9ms/step - loss: 0.0896 - accuracy: 0.9641
Epoch 5/20
314/314 [==============================] - 3s 9ms/step - loss: 0.0903 - accuracy: 0.9658
Epoch 6/20
314/314 [==============================] - 3s 9ms/step - loss: 0.0875 - accuracy: 0.9643
Epoch 7/20
314/314 [==============================] - 3s 10ms/step - loss: 0.0850 - accuracy: 0.9642
Epoch 8/20
314/314 [==============================] - 3s 11ms/step - loss: 0.0854 - accuracy: 0.9658
Epoch 9/20
314/314 [==============================] - 4s 11ms/step - loss: 0.0836 - accuracy: 0.9643 0s - loss: 0.0827 - ac - ETA: 0s - loss: 0.0832 - accuracy: 0.
Epoch 10/20
314/314 [==========

In [96]:
model.evaluate(testX_cnn,testY_cnn,verbose=2)

79/79 - 2s - loss: 0.1052 - accuracy: 0.9586


[0.10521959513425827, 0.9585987329483032]

In [99]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX_cnn)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(confusion_matrix(testY_cnn.argmax(axis=1), predIdxs))
print(classification_report(testY_cnn.argmax(axis=1), predIdxs))


[INFO] evaluating network...
[[ 986   39]
 [  65 1422]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1025
           1       0.97      0.96      0.96      1487

    accuracy                           0.96      2512
   macro avg       0.96      0.96      0.96      2512
weighted avg       0.96      0.96      0.96      2512

